# **Script oficial - Parte 1 - Organização dos dados**

In [1]:
# Abertura de biblioteca
import pandas as pd
import numpy as np
import re

# 1. Configurações Iniciais
pd.set_option('display.max_columns', 500)
file_path = '/content/BASE-ONDA-PROCESSADA--72324 - Copia.xlsx'

def carregar_e_padronizar_dados(path):
    try:
        # Lê considerando as duas primeiras linhas como cabeçalho
        df = pd.read_excel(path, header=[0, 1])

        # Função interna para unificar cabeçalhos
        def clean_header(col):
            question, option = col
            # Se a opção for "Unnamed" (vazia) ou inexistente, usa só a pergunta
            if "Unnamed" in str(option) or not str(option):
                return str(question).strip()
            # Se a pergunta for "Unnamed" (repetida/mesclada), usa só a opção
            if "Unnamed" in str(question):
                return str(option).strip()
            # Padrão: Pergunta - Opção
            return f"{str(question).strip()} - {str(option).strip()}"

        # Aplica a unificação dos nomes
        df.columns = [clean_header(col) for col in df.columns]
        print("✅ Arquivo carregado e cabeçalhos unificados com sucesso.")
        return df

    except Exception as e:
        print(f"❌ Erro ao processar arquivo: {e}")
        return None

def filtrar_respondentes_validos(df):
    # Nome exato da coluna após a unificação dos cabeçalhos
    # Verifique se o nome gerado pelo clean_header bate com este abaixo
    coluna_filtro = 'RESPOSTA ESTÁ DENTRO DA PROPORCIONALIZAÇÃO? - imported_in_delfos'

    if coluna_filtro in df.columns:
        linhas_iniciais = df.shape[0]
        # Mantém apenas quem NÃO é 'NÃO' (ou seja, mantém os válidos)
        df = df[df[coluna_filtro] != 'NÃO']
        removidos = linhas_iniciais - df.shape[0]
        print(f"✅ Filtragem de linhas: {removidos} respondentes removidos (Status 'NÃO'). Totalizando amostragem: ", df.shape[0])
    else:
        print(f"⚠️ Coluna de filtro '{coluna_filtro}' não encontrada. Nenhuma linha removida.")

    return df

def limpar_colunas_indesejadas(df):
    termos_proibidos = [
        # 1. Metadados do Sistema
        'RESPOSTA ESTÁ DENTRO DA PROPORCIONALIZAÇÃO? - imported_in_delfos',
        'respondent_id - respondent_id',
        'user_invitation_code - user_invitation_code',
        'collector_id - collector_id',
        'date_created - date_created',
        'date_modified - date_modified',
        'ip_address - ip_address',
        'status - status',
        'total_time - total_time',
        'complement_status - complement_status',
        'Você estuda ou trabalha em uma dessas atividades? #prof - Response',
        'Você estuda ou trabalha em uma dessas atividades? #prof - Outro (especifique)',

        # 2. Tags de Funil e Processamento
        '#aberta_en',
        '#awesp',     # Awareness Espontâneo
        '#aberta_op', # Perguntas abertas gerais
        '#faw',       # Funnel Awareness (Ouviu falar)
        '#fkn',       # Funnel Known (Conhece bem)
        '#flk',       # Funnel Like (Gosta)
        '#fco',       # Funnel Consideration (Consideração)
        '#fpr',       # Funnel Preference
        '#clt',       # Client (É cliente)
        '#rej',       # Rejection (Rejeição)
        '#mar',       # Marca avaliada (se quiser remover todas as colunas dessa tag)

        # 3. Perguntas de Associação/Lixo
        'PRIMEIRA PALAVRA',
        '{{',
        "PRÓXIMA COMPRA",
        'Você gostou de responder essa pesquisa? - Response'
    ]

    colunas_para_remover = []

    # Correção da sintaxe do loop aqui
    for coluna in df.columns:
        for termo in termos_proibidos:
            if termo in coluna:
                colunas_para_remover.append(coluna)
                break

    n_antes = df.shape[1]
    df_limpo = df.drop(columns=colunas_para_remover, errors='ignore')
    n_depois = df_limpo.shape[1]

    print(f"✅ Limpeza de colunas: {n_antes - n_depois} colunas removidas.")
    print(f"📊 Shape final do dataset: {df_limpo.shape}")

    return df_limpo

# --- EXECUÇÃO DO PIPELINE ---

# 1. Carregar
df = carregar_e_padronizar_dados(file_path)

if df is not None:
    # 2. Filtrar Linhas (IMPORTANTE: Fazer isso antes de remover a coluna de controle)
    df = filtrar_respondentes_validos(df)

    # 3. Limpar Colunas
    df = limpar_colunas_indesejadas(df)


✅ Arquivo carregado e cabeçalhos unificados com sucesso.
✅ Filtragem de linhas: 84 respondentes removidos (Status 'NÃO'). Totalizando amostragem:  400
✅ Limpeza de colunas: 193 colunas removidas.
📊 Shape final do dataset: (400, 128)


In [2]:
df.head(1)

,respondent_id - respondent_id,date_created - date_created,Em qual estado você vive? #est - Response,Em qual cidade você mora? #cid - Response,Em qual cidade você mora? #cid - Outro (especifique),Qual é o seu gênero ? #gen - Response,Qual é o seu gênero ? #gen - Outro (especifique),Qual é a sua idade ? #idd - Response,Qual é a sua escolaridade ? #esc - Response,Qual sua renda familiar mensal? (Renda familiar é a soma da renda de todas as pessoas que moram com você) #cls - Response,Qual das marcas abaixo você PREFERIA CONTRATAR ? Clique uma vez em cima da marca que você escolheria comprar. - Response,Qual das marcas abaixo você PREFERIA CONTRATAR ? Clique uma vez em cima da marca que você escolheria comprar.,Qual das marcas abaixo você PREFERIA CONTRATAR ? Clique uma vez em cima da marca que você escolheria comprar.,Qual das marcas abaixo você PREFERIA CONTRATAR ? Clique uma vez em cima da marca que você escolheria comprar.,Smart Fit - Response,Panobianco - Response,BodyTech - Response,"Das marcas abaixo, quais você se lembra de ter visto propaganda recentemente? Selecione todas as que lembrar. #Carac_Recall_de_Propaganda - Não me lembro de ter visto propaganda de nenhuma dessas marcas","Das marcas abaixo, quais você se lembra de ter visto propaganda recentemente? Selecione todas as que lembrar. #Carac_Recall_de_Propaganda - Smart Fit","Das marcas abaixo, quais você se lembra de ter visto propaganda recentemente? Selecione todas as que lembrar. #Carac_Recall_de_Propaganda - BlueFit","Das marcas abaixo, quais você se lembra de ter visto propaganda recentemente? Selecione todas as que lembrar. #Carac_Recall_de_Propaganda - Selfit","Das marcas abaixo, quais você se lembra de ter visto propaganda recentemente? Selecione todas as que lembrar. #Carac_Recall_de_Propaganda - Bio Ritmo","Das marcas abaixo, quais você se lembra de ter visto propaganda recentemente? Selecione todas as que lembrar. #Carac_Recall_de_Propaganda - BodyTech","Das marcas abaixo, quais você se lembra de ter visto propaganda recentemente? Selecione todas as que lembrar. #Carac_Recall_de_Propaganda - Fórmula Academia","Das marcas abaixo, quais você se lembra de ter visto propaganda recentemente? Selecione todas as que lembrar. #Carac_Recall_de_Propaganda - Cia Athletica","Das marcas abaixo, quais você se lembra de ter visto propaganda recentemente? Selecione todas as que lembrar. #Carac_Recall_de_Propaganda - Competition","Das marcas abaixo, quais você se lembra de ter visto propaganda recentemente? Selecione todas as que lembrar. #Carac_Recall_de_Propaganda - Ironberg","Das marcas abaixo, quais você se lembra de ter visto propaganda recentemente? Selecione todas as que lembrar. #Carac_Recall_de_Propaganda - Panobianco","Das marcas abaixo, quais você se lembra de ter visto propaganda recentemente? Selecione todas as que lembrar. #Carac_Recall_de_Propaganda - Pratique Fitness","Das marcas abaixo, quais você se lembra de ter visto propaganda recentemente? Selecione todas as que lembrar. #Carac_Recall_de_Propaganda - Gaviões","Das marcas abaixo, quais você se lembra de ter visto propaganda recentemente? Selecione todas as que lembrar. #Carac_Recall_de_Propaganda - Redfit","Das marcas abaixo, quais você se lembra de ter visto propaganda recentemente? Selecione todas as que lembrar. #Carac_Recall_de_Propaganda - Skyfit","Das marcas abaixo, quais você se lembra de ter visto propaganda recentemente? Selecione todas as que lembrar. #Carac_Recall_de_Propaganda - Engenharia do corpo",Com que frequência você utiliza e CONSOME CONTEÚDOS nos meios abaixo? - Televisão (canais abertos),Com que frequência você utiliza e CONSOME CONTEÚDOS nos meios abaixo? - Televisão (canais pagos),Com que frequência você utiliza e CONSOME CONTEÚDOS nos meios abaixo? - Rádio,Com que frequência você utiliza e CONSOME CONTEÚDOS nos meios abaixo? - Podcast,Com que frequência você utiliza e CONSOME CONTEÚDOS nos meios abaixo? - Instagram,Com que frequência você utiliza e CONSOME CONTEÚDOS nos meios a

# **Script oficial - Parte 2 - Limpeza e Transformação dos Dados**

In [ ]:
import pandas as pd
import numpy as np
import re

# ============================================================
# 1) AJUSTE DE CIDADE E GÊNERO
# ============================================================

df["Em qual cidade você mora? #cid - Response"] = df["Em qual cidade você mora? #cid - Response"].replace("", np.nan)
df["Em qual cidade você mora? #cid - Response"] = df["Em qual cidade você mora? #cid - Response"].fillna(
    df["Em qual cidade você mora? #cid - Outro (especifique)"]
)

df["Qual é o seu gênero ? #gen - Response"] = df["Qual é o seu gênero ? #gen - Response"].replace("", np.nan)
df["Qual é o seu gênero ? #gen - Response"] = df["Qual é o seu gênero ? #gen - Response"].fillna(
    df["Qual é o seu gênero ? #gen - Outro (especifique)"]
)

df = df.drop(columns=[
    "Em qual cidade você mora? #cid - Outro (especifique)",
    "Qual é o seu gênero ? #gen - Outro (especifique)"
])


# ============================================================
# 2) REMOÇÃO DE HTML
# ============================================================

def remove_html(text):
    if pd.isna(text):
        return text
    return re.sub(r"<.*?>", "", str(text)).strip()

#df = df.applymap(remove_html)
df = df.apply(lambda col: col.map(remove_html) if col.dtype == "object" else col)



# ============================================================
# 3) FUNÇÃO LIMPAR ESCALA LIKERT
# ============================================================

def limpar_likert(valor):
    if pd.isna(valor):
        return np.nan

    valor_str = str(valor).strip()

    if valor_str.startswith("0"):
        return 0
    if valor_str.startswith("10"):
        return 10

    if valor_str.isdigit():
        return int(valor_str)

    return np.nan


# ============================================================
# 4) IDENTIFICAR TODAS AS COLUNAS DE NOTA
# ============================================================

colunas_escala = [
    col for col in df.columns
    if "gostaria de" in col.lower()
    or "receber como presente" in col.lower()
    or "nota" in col.lower()
]

print(f"🔄 Aplicando limpeza Likert em {len(colunas_escala)} colunas...")


# ============================================================
# 5) APLICAR LIMPEZA NAS COLUNAS DE ESCALA
# ============================================================

for col in colunas_escala:
    df[col] = df[col].apply(limpar_likert)
    df[col] = pd.to_numeric(df[col], errors="coerce")
print("✅ Limpeza de escalas Likert concluída.")

# **Script Oficial - Parte 3 - Tabelas de Frequências**

In [ ]:
import json
import pandas as pd
from collections import defaultdict

def gerar_json_todas_as_tabelas(t_simples, t_multi, t_matriz, t_nota):

    resultado = {
        "perguntas_simples": [],
        "multirresposta": [],
        "matriz_texto": [],
        "matriz_nota": []
    }

    # ============================
    # 1) PERGUNTAS SIMPLES
    # ============================
    for pergunta, tabela in t_simples.items():
        resultado["perguntas_simples"].append({
            "pergunta": pergunta,
            "tabela": tabela.reset_index().rename(columns={"index": "Resposta"}).to_dict(orient="records")
        })

    # ============================
    # 2) MULTIRRESPOSTA
    # ============================
    for pergunta, tabela in t_multi.items():

        bloco = {
            "pergunta": pergunta,
            "marcas": []
        }

        for marca, row in tabela.iterrows():
            bloco["marcas"].append({
                "marca": marca,
                "frequencia_absoluta": int(row["Frequência Absoluta"]),
                "frequencia_relativa": float(row["Frequência Relativa (%)"])
            })

        resultado["multirresposta"].append(bloco)

    # ============================
    # 3) MATRIZ TEXTO
    # ============================
    for pergunta, meios in t_matriz.items():

        bloco = {
            "pergunta": pergunta,
            "itens": []
        }

        for meio, tabela in meios.items():
            bloco["itens"].append({
                "item": meio,
                "tabela": tabela.reset_index().rename(columns={"index": "Resposta"}).to_dict(orient="records")
            })

        resultado["matriz_texto"].append(bloco)

    # ============================
    # 4) MATRIZ NOTA
    # ============================
    for pergunta, marcas in t_nota.items():

        bloco = {
            "pergunta": pergunta,
            "marcas": []
        }

        for marca, tabela in marcas.items():  # <<–– AQUI FOI CORRIGIDO
            bloco["marcas"].append({
                "marca": marca,
                "tabela": tabela.reset_index().rename(columns={"index": "Nota"}).to_dict(orient="records")
            })

        resultado["matriz_nota"].append(bloco)

    return json.dumps(resultado, ensure_ascii=False, indent=2)


# ============================================================
# FUNÇÕES AUXILIARES PARA GERAÇÃO DAS TABELAS DE FREQUÊNCIA
# ============================================================

def identificar_colunas_simples(df):
    col_response = [col for col in df.columns if "response" in col.lower()]
    col_not_multi = [col for col in df.columns if " - " not in col]
    numericas = df.select_dtypes(include=["int", "float"]).columns.tolist()
    colunas_simples = list(set(col_response + col_not_multi))
    colunas_simples = [c for c in colunas_simples if c not in numericas]
    return colunas_simples

def encontrar_colunas_hifen(df):
    col_hifen = [col for col in df.columns if " - " in col]
    col_sem_response = [col for col in col_hifen if "response" not in col.lower()]
    return list(set(col_sem_response))

def agrupar_por_pergunta(colunas):
    grupos = defaultdict(list)
    for col in colunas:
        pergunta = col.split(" - ")[0].strip()
        grupos[pergunta].append(col)
    return grupos

def classificar_perguntas(df, grupos):
    grupos_multi = {}
    grupos_matriz_texto = {}
    grupos_matriz_nota = {}
    for pergunta, colunas in grupos.items():
        col_exemplo = colunas[0]
        serie = df[col_exemplo].dropna()
        if pd.api.types.is_numeric_dtype(serie):
            grupos_matriz_nota[pergunta] = colunas
            continue
        marca_exemplo = col_exemplo.split(" - ")[1].strip()
        valores = serie.astype(str).str.strip().unique()
        if marca_exemplo in valores:
            grupos_multi[pergunta] = colunas
        else:
            grupos_matriz_texto[pergunta] = colunas
    return grupos_multi, grupos_matriz_texto, grupos_matriz_nota

def tabelas_simples(df, colunas):
    tabelas = {}
    for col in colunas:
        freq_abs = df[col].value_counts(dropna=False)
        freq_rel = df[col].value_counts(normalize=True, dropna=False) * 100
        tabelas[col] = pd.DataFrame({
            "Frequência Absoluta": freq_abs,
            "Frequência Relativa (%)": freq_rel.round(1)
        })
    return tabelas

def tabelas_multiresposta(df, grupos_multi):
    tabelas = {}
    total = len(df)
    for pergunta, colunas in grupos_multi.items():
        marcas = []
        abs_list = []
        rel_list = []
        for col in colunas:
            marca = col.split(" - ")[1].strip()
            freq_abs = (df[col] == marca).sum()
            freq_rel = (freq_abs / total * 100) if total else 0
            marcas.append(marca)
            abs_list.append(freq_abs)
            rel_list.append(round(freq_rel, 1))
        tabelas[pergunta] = pd.DataFrame({
            "Frequência Absoluta": abs_list,
            "Frequência Relativa (%)": rel_list
        }, index=marcas)
    return tabelas

def tabelas_matriz_texto(df, grupos_matriz):
    tabelas = {}
    for pergunta, colunas in grupos_matriz.items():
        meios = {}
        for col in colunas:
            meio = col.split(" - ")[1].strip()
            serie = df[col].dropna().astype(str).str.strip()
            freq_abs = serie.value_counts()
            freq_rel = (serie.value_counts(normalize=True) * 100).round(1)
            meios[meio] = pd.DataFrame({
                "Frequência Absoluta": freq_abs,
                "Frequência Relativa (%)": freq_rel
            })
        tabelas[pergunta] = meios
    return tabelas

def tabelas_matriz_nota(df, grupos_matriz_nota):
    tabelas = {}
    for pergunta, colunas in grupos_matriz_nota.items():
        marcas_dict = {}
        for col in colunas:
            marca = col.split(" - ")[1].strip()
            serie = df[col].dropna()
            freq_abs = serie.value_counts().sort_index()
            freq_rel = (serie.value_counts(normalize=True).sort_index() * 100).round(1)
            marcas_dict[marca] = pd.DataFrame({
                "Frequência Absoluta": freq_abs,
                "Frequência Relativa (%)": freq_rel
            })
        tabelas[pergunta] = marcas_dict
    return tabelas

def gerar_todas_as_tabelas(df):
    col_simples = identificar_colunas_simples(df)
    t_simples = tabelas_simples(df, col_simples)
    col_hifen = encontrar_colunas_hifen(df)
    grupos = agrupar_por_pergunta(col_hifen)
    grupos_multi, grupos_matriz_texto, grupos_matriz_nota = classificar_perguntas(df, grupos)
    t_multi = tabelas_multiresposta(df, grupos_multi)
    t_matriz = tabelas_matriz_texto(df, grupos_matriz_texto)
    t_nota = tabelas_matriz_nota(df, grupos_matriz_nota)
    return t_simples, t_multi, t_matriz, t_nota


# ============================================================
# EXECUÇÃO E IMPRESSÃO DOS RESULTADOS
# ============================================================

# Chamar a função principal para gerar as tabelas
t_simples, t_multi, t_matriz, t_nota = gerar_todas_as_tabelas(df)

print("\n\n############################################################")
print("######################  RESULTADOS  ########################")
print("############################################################\n\n")


# ============================================================
# 1) PERGUNTAS SIMPLES
# ============================================================
print("\n========================================================")
print("===============  PERGUNTAS SIMPLES  ====================")
print("========================================================\n")

for pergunta, tabela in t_simples.items():
    print("\n--------------------------------------------------------")
    print("PERGUNTA:", pergunta)
    print("--------------------------------------------------------")
    print(tabela)
    print("\n")


# ============================================================
# 2) MULTIRRESPOSTA
# ============================================================
print("\n========================================================")
print("=================  MULTIRRESPOSTA  =====================")
print("========================================================\n")

for pergunta, tabela in t_multi.items():
    print("\n--------------------------------------------------------")
    print("PERGUNTA:", pergunta)
    print("--------------------------------------------------------")
    print(tabela)
    print("\n")


# ============================================================
# 3) MATRIZ TEXTO
# ============================================================
print("\n========================================================")
print("==================  MATRIZ TEXTO  =======================")
print("========================================================\n")

for pergunta, meios in t_matriz.items():
    print("\n--------------------------------------------------------")
    print("PERGUNTA:", pergunta)
    print("--------------------------------------------------------")

    for meio, tabela in meios.items():
        print(f"\n--- {meio} ---")
        print(tabela)
        print("\n")


# ============================================================
# 4) MATRIZ NOTA (0–10)
# ============================================================
print("\n========================================================")
print("===================  MATRIZ NOTA  =======================")
print("========================================================\n")

for pergunta, marcas in t_nota.items():
    print("\n--------------------------------------------------------")
    print("PERGUNTA:", pergunta)
    print("--------------------------------------------------------")

    for marca, tabela in marcas.items():
        print(f"\n--- {marca} ---")
        print(tabela)
        print("\n")

# ============================================================
# SALVAR RESULTADOS EM VARIÁVEL E EM ARQUIVO JSON
# ============================================================

resultado_json = gerar_json_todas_as_tabelas(t_simples, t_multi, t_matriz, t_nota)

# salvar localmente
with open("resultado_pesquisa.json", "w", encoding="utf-8") as f:
    f.write(resultado_json)

print("Arquivo JSON salvo como resultado_pesquisa.json")
